**Modelos de clasificación de texto**

In [ ]:
from google.colab import drive, files

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import warnings
from google.colab import files
import io



if not sys.warnoptions:
  warnings.simplefilter("ignore")

In [ ]:
# 1. Base de Vacantes
vacantes = pd.read_pickle('/content/drive/MyDrive/Datos/base1_vacantes.pkl')
# 2. Base de Clasificaciones
clasificacion = pd.read_pickle('/content/drive/MyDrive/Datos/clasificacion2.pkl')


In [ ]:
vacantes.head()

,id,fecha,titulo,descripcion
0,1884375002,02jan2020,analista sst licencia,empresa telecomunicaciones requiere equipo tr...
1,1884375149,03jan2020,desarrollador r as medellian,requiere tecnologo sistemas afines estudiante...
2,1884374755,02jan2020,asesor comercial bachiller,entidad financiera encuentra busqueda person...
3,1884376204,03jan2020,supervisor cobranza,outsourcing cobranza requiere tecnico estudia...
4,1884376605,03jan2020,atencion cliente,empresa sector busca personas nivel ingles a...


In [ ]:
clasificacion.head()

,id,descripcion,titulo,teletrabajo
0,11-1011.00,determinan formulan politicas cumplen funcione...,directores ejecutivos,1
1,11-1011.03,comunican coordinan gerencia accionistas clie...,directores desarrollo sostenible,1
2,11-1021.00,planifican dirigen coordinan operaciones organ...,gerentes generales operaciones,1
3,11-2011.00,planifican dirigen coordinan politicas program...,gerentes publicidad promociones,1
4,11-2021.00,planifican dirigen coordinan politicas program...,gerentes mercadeo,1


In [ ]:
stop_words = ['empresa', 'entidad', 'multinacional','importante', 
               'prestigioso', 'prestigiosa','reconocida', 'bogota',
               'medellin', 'barranquilla', 'santa marta', 'cartagena', 
               'tunja', 'pasto','neiva','monteria','bucaramanga','girardot',
               'ibague','popayan','villavicencio','cucuta','valledupar','leticia',
               'manizales','yopal','quibdo','riohacha','pasto','armenia','pereira','sincelejo',
               'cundinamarca','valle','antioquia','atlantico','bolivar',
               'valle del cauca','cauca','santander','boyaca','norte de santander',
               'cesar','tolima','nariño','narino','magdalena','cordoba','guajira',
               'choco','san andres','huila','vaupes','guainia','amazonas','arauca',
               'vichada','quindio','risaralda','caldas','caqueta','casanare','meta',
               'putumayo','sucre','guaviare','smlv','hoja de vida','medellian', 'equipo trabajo',
               'semanales','mensuales','destacada','mañana','fin semana','semana',
               'mianimo','afines','experiencia laboral','experiencia','personas','trabajar horas',
               'requiere','norte','occidente','nivel ingles avanzado',
               'nivel ingles intermedio','nivel ingles basico','nivel ingles','ingles','mianima','bachiller','tecnologos',
               'carreras','carrera','estudiantes','estudiante','ciudad','tecnicas','tecnicos','tecnologo']

vacantes['descripcion2'] = vacantes['descripcion']
vacantes['titulo2'] = vacantes['titulo']

for word in stop_words:
    vacantes['descripcion2'] = vacantes['descripcion2'].str.replace(word,'')
    vacantes['titulo2'] = vacantes['titulo2'].str.replace(word,'')

In [ ]:
vacantes['descripcion2'] = vacantes['descripcion2'].str.strip()
vacantes['titulo2'] = vacantes['titulo2'].str.strip()

In [ ]:
espacios = ['  ', '   ','    ']
vacantes['descripcion3'] = vacantes['descripcion2']
vacantes['titulo3'] = vacantes['titulo2']

for espacio in espacios:
    vacantes['descripcion3'] = vacantes['descripcion3'].str.replace(espacio,' ')
    vacantes['titulo3'] = vacantes['titulo3'].str.replace(espacio,' ')


In [ ]:
print('Número de vacantes:',vacantes['id'].nunique())
print('Número de ocupaciones:',clasificacion['id'].nunique())

Número de vacantes: 794760
Número de ocupaciones: 721


In [ ]:
vacantes.head()

,id,fecha,titulo,descripcion,descripcion2,titulo2,descripcion3,titulo3
0,1884375002,02jan2020,analista sst licencia,empresa telecomunicaciones requiere equipo tr...,telecomunicaciones analista seguridad salud...,analista sst licencia,telecomunicaciones analista seguridad salud t...,analista sst licencia
1,1884375149,03jan2020,desarrollador r as medellian,requiere tecnologo sistemas afines estudiante...,sistemas ingenieria sistemas adelante posea...,desarrollador r as,sistemas ingenieria sistemas adelante posea ...,desarrollador r as
2,1884374755,02jan2020,asesor comercial bachiller,entidad financiera encuentra busqueda person...,financiera encuentra busqueda personal desee ...,asesor comercial,financiera encuentra busqueda personal desee o...,asesor comercial
3,1884376204,03jan2020,supervisor cobranza,outsourcing cobranza requiere tecnico estudia...,outsourcing cobranza tecnico administrativa...,supervisor cobranza,outsourcing cobranza tecnico administrativas ...,supervisor cobranza
4,1884376605,03jan2020,atencion cliente,empresa sector busca personas nivel ingles a...,sector busca lianeas atencion cliente m...,atencion cliente,sector busca lianeas atencion cliente manejan...,atencion cliente


In [ ]:
pip install pytextdist


In [ ]:
from pytextdist.edit_distance import levenshtein_distance, levenshtein_similarity

In [ ]:
def m_distancia(x, lista2):
    mejor_match = None
    mejor_levenshtein = 10000
    #for descripcion in vacantes['descripcion']:
    for string in lista2:
        score_actual = levenshtein_distance(string,x)
        if score_actual < mejor_levenshtein:
            mejor_levenshtein = score_actual
            mejor_match = string
    return mejor_levenshtein, mejor_match

def m_similaridad(x, lista2):
    mejor_match = None
    mejor_similaridad = 0
    #for descripcion in vacantes['descripcion']:
    for string in lista2:
        score_actual = levenshtein_similarity(string,x)
        if score_actual > mejor_similaridad:
            mejor_similaridad = score_actual
            mejor_match = string
    return mejor_similaridad, mejor_match

In [ ]:
vacantes1 = vacantes[0:10000]
#clasificacion1 = clasificacion[0:39]
lista1 = vacantes1['titulo3'].tolist()
lista2 = clasificacion['titulo'].tolist()
#surveys_df[['plot_id', 'species_id']]
#lista1 = vacantes1[['id','descripcion']].tolist()
#lista2 = clasificacion1['descripcion'].tolist()

In [ ]:
base_similaridad = []
variables = ['resultado_similaridad','descripcion']

for string1 in lista1:
    resultado = m_similaridad(string1, lista2)
    descripcion = string1
    base_similaridad.append((resultado, descripcion))

In [ ]:
base_distancia = []
variables = ['resultado_distancia','descripcion']
#variables = ['puntaje_distancia','id','codigo','descripcion','clasificacion']

#for x in clasificacion[]
for string1 in lista1:
    resultado = m_distancia(string1, lista2)
    #print("Descripción: ", string1, "Puntaje de emparejamiento:", resultado)
    #id = lista1['id']
    #codigo = lista2['id']
    descripcion = string1
    #clasificacion = lista2[
    base_distancia.append((resultado, descripcion))
    #base_distancia.append((resultado, id, codigo, descripcion, clasificacion))

In [ ]:
df_distancia = pd.DataFrame(base_distancia, columns=variables)
df_similaridad = pd.DataFrame(base_similaridad, columns=variables)

In [ ]:
df_similaridad.head()

,resultado_similaridad,descripcion
0,"(0.8095238095238095, analistas inteligencia)",analista sst licencia
1,"(0.5625, derribadores)",desarrollador r as
2,"(0.5625, buzos comerciales)",asesor comercial
3,"(0.55, supervisores aviacion)",supervisor cobranza
4,"(0.41666666666666663, empleados estacionamiento)",atencion cliente


In [ ]:
df_distancia.head()

,resultado_similaridad,descripcion
0,"(4, analistas inteligencia)",analista sst licencia
1,"(7, derribadores)",desarrollador r as
2,"(7, buzos comerciales)",asesor comercial
3,"(9, supervisores aviacion)",supervisor cobranza
4,"(10, economistas)",atencion cliente


In [ ]:
df_distancia.to_csv('distancia.txt', index = False)
df_similaridad.to_csv('similaridad.txt', index = False)
files.download('distancia.txt') 
files.download('similaridad.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Jaro & Jaro-Winkler**

In [ ]:
from pytextdist.edit_distance import jaro_similarity, jaro_winkler_similarity

str_a = 'sitten'
str_b = 'sitting'
simi_j = jaro_similarity(str_a,str_b)
simi_jw = jaro_winkler_similarity(str_a,str_b)
print(f"Jaro Similarity:{simi_j:.2f}\nJaro-Winkler Similarity:{simi_jw:.2f}")


Jaro Similarity:0.85
Jaro-Winkler Similarity:0.91


In [ ]:
def jaro(x, lista2):
    mejor_match = None
    mejor_jaro = 0
    #for descripcion in vacantes['descripcion']:
    for string in lista2:
        score_actual = jaro_similarity(string,x)
        if score_actual > mejor_jaro:
            mejor_jaro = score_actual
            mejor_match = string
    return mejor_jaro, mejor_match

def jaro_winkler(x, lista2):
    mejor_match = None
    mejor_jaro_w = 0
    #for descripcion in vacantes['descripcion']:
    for string in lista2:
        score_actual = jaro_winkler_similarity(string,x)
        if score_actual > mejor_jaro_w:
            mejor_jaro_w = score_actual
            mejor_match = string
    return mejor_jaro_w, mejor_match

In [ ]:
base_jaro = []
variables = ['resultado_jaro','descripcion']

for string1 in lista1:
    resultado = jaro(string1, lista2)
    descripcion = string1
    base_jaro.append((resultado, descripcion))

In [ ]:
base_jarow = []
variables = ['resultado_jarow','descripcion']

for string1 in lista1:
    resultado = jaro_winkler(string1, lista2)
    descripcion = string1
    base_jarow.append((resultado, descripcion))

In [ ]:
df_jaro = pd.DataFrame(base_jaro, columns=variables)
df_jarow = pd.DataFrame(base_jarow, columns=variables)

In [ ]:
df_jaro

,resultado_jarow,descripcion
0,"(0.8478313840155945, analistas logistica)",analista sst licencia
1,"(0.7513888888888888, grabadores talladores)",desarrollador r as
2,"(0.7491452991452991, inspectores agricolas)",asesor comercial
3,"(0.8055555555555557, supervisores aviacion)",supervisor cobranza
4,"(0.734920634920635, comerciantes internet)",atencion cliente
5,"(0.8319444444444445, tecnicos robotica)",tecnico soporte sitio ocaña
6,"(0.7236363636363636, planificadores renovacion...",gran convocatoria agente bilingue
7,"(0.6721256684491977, tecnicos avionica)",ejecutivo cobranza
8,"(0.7352118748460211, trabajadores aplicacion t...",asesor comercial externo libranza hipotecario


In [ ]:
df_jarow

,resultado_jarow,descripcion
0,"(0.9239156920077973, analistas logistica)",analista sst licencia
1,"(0.7574074074074073, derribadores)",desarrollador r as
2,"(0.8722222222222222, asesores credito)",asesor comercial
3,"(0.9027777777777779, supervisores aviacion)",supervisor cobranza
4,"(0.734920634920635, comerciantes internet)",atencion cliente
5,"(0.9159722222222222, tecnicos robotica)",tecnico soporte sitio ocaña
6,"(0.7351893408134643, gerentes instalacion ener...",gran convocatoria agente bilingue
7,"(0.6721256684491977, tecnicos avionica)",ejecutivo cobranza
8,"(0.8254951011048572, asesores credito)",asesor comercial externo libranza hipotecario


In [ ]:
df_jaro.to_csv('jaro.txt', index = False)
df_jarow.to_csv('jarow.txt', index = False)
files.download('jaro.txt') 
files.download('jarow.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Cosine Similarity**

In [ ]:
from pytextdist.vector_similarity import cosine_similarity

phrase_a = 'For Paperwork Reduction Act Notice, see your tax return instructions. For Paperwork Reduction Act Notice, see your tax return instructions.'
phrase_b = 'For Disclosure, Privacy Act, and Paperwork Reduction Act Notice, see separate instructions. Form 1040'
simi_1 = cosine_similarity(phrase_a, phrase_b, n=1)
simi_2 = cosine_similarity(phrase_a, phrase_b, n=2)
print(f"Unigram Cosine Similarity:{simi_1:.2f}\nBigram Cosine Similarity:{simi_2:.2f}")


Unigram Cosine Similarity:0.65
Bigram Cosine Similarity:0.38


In [ ]:
def cosine1(x, lista2):
    mejor_match = None
    mejor_cosine1 = 0
    #for descripcion in vacantes['descripcion']:
    for string in lista2:
        score_actual = cosine_similarity(string,x,n=1)
        if score_actual > mejor_cosine1:
            mejor_cosine1 = score_actual
            mejor_match = string
    return mejor_cosine1, mejor_match

def cosine2(x, lista2):
    mejor_match = None
    mejor_cosine2 = 0
    #for descripcion in vacantes['descripcion']:
    for string in lista2:
        score_actual = cosine_similarity(string,x,n=2)
        if score_actual > mejor_cosine2:
            mejor_cosine2 = score_actual
            mejor_match = string
    return mejor_cosine2, mejor_match

In [ ]:
base_cosine1 = []
variables = ['resultado_cosine1','descripcion']

for string1 in lista1:
    resultado = cosine1(string1, lista2)
    descripcion = string1
    base_cosine1.append((resultado, descripcion))

In [ ]:
base_cosine2 = []
variables = ['resultado_cosine2','descripcion']

for string1 in lista1:
    resultado = cosine2(string1, lista2)
    descripcion = string1
    base_cosine2.append((resultado, descripcion))

In [ ]:
df_cosine1 = pd.DataFrame(base_cosine1, columns=variables)
df_cosine2 = pd.DataFrame(base_cosine2, columns=variables)

In [ ]:
df_cosine1

,resultado_cosine2,descripcion
0,"(0.2886751345948129, enfermeros practicos voca...",analista sst licencia
1,"(0, None)",desarrollador r as
2,"(0.40824829046386296, planificadores continuid...",asesor comercial
3,"(0, None)",supervisor cobranza
4,"(0.40824829046386296, representantes servicio ...",atencion cliente
5,"(0.2886751345948129, tecnico procesamiento sem...",tecnico soporte sitio ocaña
6,"(0, None)",gran convocatoria agente bilingue
7,"(0, None)",ejecutivo cobranza
8,"(0.2581988897471611, planificadores continuida...",asesor comercial externo libranza hipotecario


In [ ]:
df_cosine2

,resultado_cosine2,descripcion


In [ ]:
df_cosine1.to_csv('cosine1.txt', index = False)
df_cosine2.to_csv('cosine2.txt', index = False)
files.download('cosine1.txt') 
files.download('cosine2.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Jaccard Similarity**

In [ ]:
from pytextdist.vector_similarity import jaccard_similarity

phrase_a = 'For Paperwork Reduction Act Notice, see your tax return instructions. For Paperwork Reduction Act Notice, see your tax return instructions.'
phrase_b = 'For Disclosure, Privacy Act, and Paperwork Reduction Act Notice, see separate instructions. Form 1040'
simi_1 = jaccard_similarity(phrase_a, phrase_b, n=1)
simi_2 = jaccard_similarity(phrase_a, phrase_b, n=2)
print(f"Unigram Jaccard Similarity:{simi_1:.2f}\nBigram Jaccard Similarity:{simi_2:.2f}")

Unigram Jaccard Similarity:0.47
Bigram Jaccard Similarity:0.22


In [ ]:
def jaccard1(x, lista2):
    mejor_match = None
    mejor_jaccard1 = 0
    #for descripcion in vacantes['descripcion']:
    for string in lista2:
        score_actual = jaccard_similarity(string,x,n=1)
        if score_actual > mejor_jaccard1:
            mejor_jaccard1 = score_actual
            mejor_match = string
    return mejor_jaccard1, mejor_match

def jaccard2(x, lista2):
    mejor_match = None
    mejor_jaccard2 = 0
    #for descripcion in vacantes['descripcion']:
    for string in lista2:
        score_actual = jaccard_similarity(string,x,n=2)
        if score_actual > mejor_jaccard2:
            mejor_jaccard2 = score_actual
            mejor_match = string
    return mejor_jaccard2, mejor_match

In [ ]:
base_jaccard1 = []
variables = ['resultado_jaccard1','descripcion']

for string1 in lista1:
    resultado = jaccard1(string1, lista2)
    descripcion = string1
    base_jaccard1.append((resultado, descripcion))

In [ ]:
base_jaccard2 = []
variables = ['resultado_jaccard2','descripcion']

for string1 in lista1:
    resultado = jaccard2(string1, lista2)
    descripcion = string1
    base_jaccard2.append((resultado, descripcion))

In [ ]:
df_jaccard1 = pd.DataFrame(base_jaccard1, columns=variables)
df_jaccard2 = pd.DataFrame(base_jaccard2, columns=variables)

In [ ]:
df_jaccard1

,resultado_jaccard2,descripcion
0,"(0.16666666666666666, enfermeros practicos voc...",analista sst licencia
1,"(0, None)",desarrollador r as
2,"(0.25, planificadores continuidad comercial)",asesor comercial
3,"(0, None)",supervisor cobranza
4,"(0.25, representantes servicio cliente)",atencion cliente
5,"(0.16666666666666666, tecnico procesamiento se...",tecnico soporte sitio ocaña
6,"(0, None)",gran convocatoria agente bilingue
7,"(0, None)",ejecutivo cobranza
8,"(0.14285714285714285, planificadores continuid...",asesor comercial externo libranza hipotecario


In [ ]:
df_jaccard2

,resultado_jaccard2,descripcion


In [ ]:
df_jaccard1.to_csv('jaccard1.txt', index = False)
df_jaccard2.to_csv('jaccard2.txt', index = False)
files.download('jaccard1.txt') 
files.download('jaccard2.txt')

**Sorensen Dice Similarity**

In [ ]:
from pytextdist.vector_similarity import sorensen_dice_similarity

phrase_a = 'For Paperwork Reduction Act Notice, see your tax return instructions. For Paperwork Reduction Act Notice, see your tax return instructions.'
phrase_b = 'For Disclosure, Privacy Act, and Paperwork Reduction Act Notice, see separate instructions. Form 1040'
simi_1 = sorensen_dice_similarity(phrase_a, phrase_b, n=1)
simi_2 = sorensen_dice_similarity(phrase_a, phrase_b, n=2)
print(f"Unigram Sorensen Dice Similarity:{simi_1:.2f}\nBigram Sorensen Dice Similarity:{simi_2:.2f}")

Unigram Sorensen Dice Similarity:0.64
Bigram Sorensen Dice Similarity:0.36


In [ ]:
def sore1(x, lista2):
    mejor_match = None
    mejor_sore1 = 0
    #for descripcion in vacantes['descripcion']:
    for string in lista2:
        score_actual = sorensen_dice_similarity(string,x,n=1)
        if score_actual > mejor_sore1:
            mejor_sore1 = score_actual
            mejor_match = string
    return mejor_sore1, mejor_match

In [ ]:
def sore2(x, lista2):
    mejor_match = None
    mejor_sore2 = 0
    #for descripcion in vacantes['descripcion']:
    for string in lista2:
        score_actual = sorensen_dice_similarity(string,x,n=2)
        if score_actual > mejor_sore2:
            mejor_sore2 = score_actual
            mejor_match = string
    return mejor_sore2, mejor_match

In [ ]:
base_sore1 = []
variables = ['resultado_sore1','descripcion']

for string1 in lista1:
    resultado = sore1(string1, lista2)
    descripcion = string1
    base_sore1.append((resultado, descripcion))

In [ ]:
base_sore2 = []
variables = ['resultado_sore2','descripcion']

for string1 in lista1:
    resultado = sore2(string1, lista2)
    descripcion = string1
    base_sore2.append((resultado, descripcion))

In [ ]:
df_sore1 = pd.DataFrame(base_sore1, columns=variables)

In [ ]:
df_sore2 = pd.DataFrame(base_sore2, columns=variables)

In [ ]:
df_sore1

,resultado_sore1,descripcion
0,"(0.2857142857142857, enfermeros practicos voca...",analista sst licencia
1,"(0, None)",desarrollador r as
2,"(0.4, planificadores continuidad comercial)",asesor comercial
3,"(0, None)",supervisor cobranza
4,"(0.4, representantes servicio cliente)",atencion cliente
5,"(0.2857142857142857, tecnico procesamiento sem...",tecnico soporte sitio ocaña
6,"(0, None)",gran convocatoria agente bilingue
7,"(0, None)",ejecutivo cobranza
8,"(0.25, planificadores continuidad comercial)",asesor comercial externo libranza hipotecario


In [ ]:
df_sore1.to_csv('sore1.txt', index = False)
df_sore2.to_csv('sore2.txt', index = False)
files.download('sore1.txt') 
files.download('sore2.txt')

**Q-gram similarity**

In [ ]:
from pytextdist.vector_similarity import qgram_similarity

phrase_a = 'For Paperwork Reduction Act Notice, see your tax return instructions. For Paperwork Reduction Act Notice, see your tax return instructions.'
phrase_b = 'For Disclosure, Privacy Act, and Paperwork Reduction Act Notice, see separate instructions. Form 1040'
simi_1 = qgram_similarity(phrase_a, phrase_b, n=1)
simi_2 = qgram_similarity(phrase_a, phrase_b, n=2)
print(f"Unigram Q-Gram Similarity:{simi_1:.2f}\nBigram Q-Gram Similarity:{simi_2:.2f}")

Unigram Q-Gram Similarity:0.32
Bigram Q-Gram Similarity:0.15


In [ ]:
def qgram1(x, lista2):
    mejor_match = None
    mejor_qgram1 = 0
    #for descripcion in vacantes['descripcion']:
    for string in lista2:
        score_actual = qgram_similarity(string,x,n=1)
        if score_actual > mejor_qgram1:
            mejor_qgram1 = score_actual
            mejor_match = string
    return mejor_qgram1, mejor_match

In [ ]:
def qgram2(x, lista2):
    mejor_match = None
    mejor_qgram2 = 0
    #for descripcion in vacantes['descripcion']:
    for string in lista2:
        score_actual = qgram_similarity(string,x,n=2)
        if score_actual > mejor_qgram2:
            mejor_qgram2 = score_actual
            mejor_match = string
    return mejor_qgram2, mejor_match

In [ ]:
base_qgram1 = []
variables = ['resultado_qgram1','descripcion']

for string1 in lista1:
    resultado = qgram1(string1, lista2)
    descripcion = string1
    base_qgram1.append((resultado, descripcion))

In [ ]:
base_qgram2 = []
variables = ['resultado_qgram2','descripcion']

for string1 in lista1:
    resultado = qgram2(string1, lista2)
    descripcion = string1
    base_qgram2.append((resultado, descripcion))

In [ ]:
df_qgram1 = pd.DataFrame(base_qgram1, columns=variables)

In [ ]:
df_qgram2 = pd.DataFrame(base_qgram2, columns=variables)

In [ ]:
df_qgram1

,resultado_qgram2,descripcion
0,"(0.16666666666666663, enfermeros practicos voc...",analista sst licencia
1,"(0, None)",desarrollador r as
2,"(0.25, planificadores continuidad comercial)",asesor comercial
3,"(0, None)",supervisor cobranza
4,"(0.25, representantes servicio cliente)",atencion cliente
5,"(0.16666666666666663, tecnico procesamiento se...",tecnico soporte sitio ocaña
6,"(0, None)",gran convocatoria agente bilingue
7,"(0, None)",ejecutivo cobranza
8,"(0.1428571428571429, planificadores continuida...",asesor comercial externo libranza hipotecario


In [ ]:
df_qgram2

,resultado_qgram2,descripcion


In [ ]:
df_qgram1.to_csv('qgram1.txt', index = False)
df_qgram2.to_csv('qgram2.txt', index = False)
files.download('qgram1.txt') 
files.download('qgram2.txt')

**Bag of Words Model**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(lista1)
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
vocab = cv.get_feature_names()

In [ ]:
pd.DataFrame(cv_matrix, columns=vocab)

,aaplica,aarma,abrasivos,aburrarequisitos,abuscas,academica,academico,academicocondiciones,academicorequisitos,academicos,acadi,acceso,access,accidentes,accionr,acercarse,acero,acompai,acordado,acorde,acordes,acostumbradas,acta,actitud,actividades,activos,actuales,actualizada,actualizadar,acuerdo,acuerdos,adaptacion,adecuado,adelante,adhesivos,adicionalmente,administracion,administrador,administrando,administrar,...,vida,vidar,vigencia,vigente,vigentecon,vigilancia,vii,vinculacion,vincular,virtual,virtualesconocimiento,vision,vitrina,viva,vive,vivir,vocabulario,vuelo,way,web,windowsr,word,wordexcel,workforce,world,xa,xacumplir,xaofrecer,xaxaxaxaconocimiento,xaxaxaxadiseño,xaxaxaxarediseño,xaxaxaxaxadiseño,xaxaxaxaxaxaxaxa,xaxaxaxaxaxaxaxaxaxaxaxaxaxaxaxaxa,yenifer,yoprofesionales,yumbo,zarzamora,zona,zonas
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
95,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
bv = CountVectorizer(ngram_range=(2,2))
bv_matrix = bv.fit_transform(lista1)

bv_matrix = bv_matrix.toarray()
vocab = bv.get_feature_names()
pd.DataFrame(bv_matrix, columns=vocab)

,aaplica ahora,aarma horariocontrato,abrasivos adhesivos,aburrarequisitos minima,abuscas ser,academica academico,academica culminado,academica debe,academica es,academica esperamosr,academica ingeniero,academica profesionales,academico academicocondiciones,academico culminado,academico laborales,academico libreta,academico lugar,academico realizacion,academico tecnico,academicocondiciones laboraleshorarios,academicorequisitos academico,academicos proceso,acadi mica,acadi micos,acceso grandes,access excel,accidentes reintegro,accionr fecha,acercarse hoja,acero pinturas,acompai amiento,acordado lograr,acorde indispensable,acorde neio,acordes perfil,acostumbradas manejar,acta diploma,actitud cambio,actitud servicio,actividades acordes,...,xa xacumplir,xa xaxaxaxaxadiseño,xa xaxaxaxaxaxaxaxaxaxaxaxaxaxaxaxaxa,xacumplir asignadas,xacumplir estandares,xaofrecer informacion,xaxaxaxaconocimiento tendencias,xaxaxaxadiseño implementacion,xaxaxaxarediseño productos,xaxaxaxaxadiseño creacion,xaxaxaxaxaxaxaxa adaptacion,xaxaxaxaxaxaxaxa amabilidad,xaxaxaxaxaxaxaxa amigable,xaxaxaxaxaxaxaxa capacidad,xaxaxaxaxaxaxaxa confiable,xaxaxaxaxaxaxaxa diseñar,xaxaxaxaxaxaxaxa disposicion,xaxaxaxaxaxaxaxa ejecutar,xaxaxaxaxaxaxaxa habilidades,xaxaxaxaxaxaxaxa implementacion,xaxaxaxaxaxaxaxa innovacion,xaxaxaxaxaxaxaxa juntos,xaxaxaxaxaxaxaxa organizacion,xaxaxaxaxaxaxaxa orientacion,xaxaxaxaxaxaxaxa realizar,xaxaxaxaxaxaxaxa relaciones,xaxaxaxaxaxaxaxa sencillo,xaxaxaxaxaxaxaxa trabajo,xaxaxaxaxaxaxaxa vision,xaxaxaxaxaxaxaxaxaxaxaxaxaxaxaxaxa deseable,xaxaxaxaxaxaxaxaxaxaxaxaxaxaxaxaxa inscribirme,xaxaxaxaxaxaxaxaxaxaxaxaxaxaxaxaxa presentar,xaxaxaxaxaxaxaxaxaxaxaxaxaxaxaxaxa tener,yenifer rodriguez,yoprofesionales call,zarzamora pregunta,zona industrial,zona influencia,zona pago,zonas eje
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
95,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**TF-IDF Model**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(lista1)
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

,aaplica,aarma,abrasivos,aburrarequisitos,abuscas,academica,academico,academicocondiciones,academicorequisitos,academicos,acadi,acceso,access,accidentes,accionr,acercarse,acero,acompai,acordado,acorde,acordes,acostumbradas,acta,actitud,actividades,activos,actuales,actualizada,actualizadar,acuerdo,acuerdos,adaptacion,adecuado,adelante,adhesivos,adicionalmente,administracion,administrador,administrando,administrar,...,vida,vidar,vigencia,vigente,vigentecon,vigilancia,vii,vinculacion,vincular,virtual,virtualesconocimiento,vision,vitrina,viva,vive,vivir,vocabulario,vuelo,way,web,windowsr,word,wordexcel,workforce,world,xa,xacumplir,xaofrecer,xaxaxaxaconocimiento,xaxaxaxadiseño,xaxaxaxarediseño,xaxaxaxaxadiseño,xaxaxaxaxaxaxaxa,xaxaxaxaxaxaxaxaxaxaxaxaxaxaxaxaxa,yenifer,yoprofesionales,yumbo,zarzamora,zona,zonas
0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.16,0.0,0.0,0.0,0.00,0.0,0.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.35,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.15,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.09,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.08,0.0,0.0,0.0,0.0,0.08,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,..

In [ ]:
tv_matrix2 = tv.fit_transform(lista2)
tv_matrix2 = tv_matrix2.toarray()

vocab2 = tv.get_feature_names()
pd.DataFrame(np.round(tv_matrix2, 2), columns=vocab)

**Lemanización**

In [ ]:
import spacy
import spacy.cli
from spacy import displacy
from spacy.lang.es.examples import sentences
spacy.cli.download("es_core_news_md")
#from spacy import es_core_news_sm
#!pip download spacy


In [ ]:
import spacy
import spacy_spanish_lemmatizer
# Change "es" to the Spanish model installed in step 2
nlp = spacy.load("es")
nlp.replace_pipe("lemmatizer", "spanish_lemmatizer")
for token in nlp(
    """Con estos fines, la Dirección de Gestión y Control Financiero monitorea
       la posición de capital del Banco y utiliza los mecanismos para hacer un
       eficiente manejo del capital."""
):
    print(token.text, token.lemma_)

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

sentence = "He was running and eating at same time. He has bad habit of swimming after playing long hours in the Sun."
punctuations="?:!.,;"
sentence_words = nltk.word_tokenize(sentence)
for word in sentence_words:
    if word in punctuations:
        sentence_words.remove(word)

sentence_words
print("{0:20}{1:20}".format("Word","Lemma"))
for word in sentence_words:
    print ("{0:20}{1:20}".format(word,wordnet_lemmatizer.lemmatize(word)))

In [ ]:
nlp = spacy.load('es_core_news_md')

In [ ]:
nrows = len(vacantes1)

In [ ]:
lista_texto_lematizado = []

for row in range(0,nrows):
  lista_lematizada = []
  texto = vacantes1.loc[row]['descripcion3']
  type(texto)
  #texto = vacantes1['descripcion']
  texto_palabras = texto.split(" ")
  for palabra in texto_palabras:
    #palabra2 = nlp(palabra)
    lista_lematizada.append(nlp(palabra))

    #for i in lista_lematizada:
      #print(i.lemma_,palabra.text)
    #print(palabra,palabra2,lista_lematizada)
    #palabra = nlp(i)
    #print(palabra.text,palabra.lemma_)
  #texto_lematizado = " ".join(lista_lematizada)
  texto_lematizado = " ".join(map(str, lista_lematizada))
  #for word in texto_lematizado:
    #print(word,palabra)
  #print(texto_lematizado)
  lista_texto_lematizado.append(texto_lematizado)
  #print(lista_texto_lematizado)

In [ ]:
nlp = spacy.load('es_core_news_md')

In [ ]:
from spacy.lang.es.stop_words import STOP_WORDS
stopwords_spacy = list(STOP_WORDS)
print(stopwords_spacy)
len(stopwords_spacy)

In [ ]:
nltk.download('stopwords')

In [ ]:
import nltk
from nltk.corpus import stopwords
stop_words_sp = set(stopwords.words('spanish'))
print(stop_words_sp)
len(stop_words_sp)

In [ ]:
for token in doc:
    if token.is_stop == False:
        print(token)

In [ ]:
for lem in doc:
    print(lem.text, lem.lemma_)

In [ ]:
def lemanizacion(x, lista_n):
  for descripcion in lista_n:
    doc = nlp(descripcion)
    for word in doc:
      print(word.lemma)
      return word.lemma_

In [ ]:
for x in lista1:
  palabra = lemanizacion(x,lista1)


In [ ]:
nlp = spacy.load('es_core_news_md')

In [ ]:
nrows = len(vacantes1)

In [ ]:
lista_texto_lematizado = []

for row in range(0,nrows):
  lista_lematizada = []
  texto = vacantes1.loc[row]['descripcion']
  type(texto)
  #texto = vacantes1['descripcion']
  texto_palabras = texto.split(" ")
  for palabra in texto_palabras:
    #palabra2 = nlp(palabra)
    lista_lematizada.append(nlp(palabra))
    #for i in lista_lematizada:
      #print(i.lemma_,palabra.text)
    #print(palabra,palabra2,lista_lematizada)
    #palabra = nlp(i)
    #print(palabra.text,palabra.lemma_)
  #texto_lematizado = " ".join(lista_lematizada)
  texto_lematizado = " ".join(map(str, lista_lematizada))
  #print(texto_lematizado)
  lista_texto_lematizado.append(texto_lematizado)
  #print(lista_texto_lematizado)

In [ ]:
vacantes1['parsed'] = lista_texto_lematizado